<a href="https://colab.research.google.com/github/RickyDoan/DL-TF-Potato-Leaf-Images-Classification-Prediction/blob/main/DL_TF_Potato_leaf_images_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
# # prompt: write the code to unzip a file

# import zipfile

# # Replace 'your_zip_file.zip' with the actual name of your zip file
# with zipfile.ZipFile('/content/drive/MyDrive/A Tensor Flow/Tomato-leaf-desease-detection_copy/PlantVillage.zip', 'r') as zip_ref:
#     zip_ref.extractall('/content/drive/MyDrive/A Tensor Flow/Tomato-leaf-desease-detection_copy/PlantVillage') # Replace with desired extraction path


In [ ]:
image_size = (256,256)
batch_size = 32

In [ ]:
data_dir = '/content/drive/MyDrive/A Tensor Flow/Tomato-leaf-desease-detection_copy/PlantVillage/PlantVillage'

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/A Tensor Flow/Tomato-leaf-desease-detection_copy/PlantVillage/PlantVillage',
    seed = 123,
    shuffle = True,
    image_size = (256,256),
    batch_size = 32
)

In [ ]:
dataset.class_names

In [ ]:
for image_batch, labels_batch in dataset.take(1):
    print(image_batch.shape)
    print(labels_batch.numpy())

In [ ]:
plt.figure(figsize=(10,10))
for i in range(12):
    ax = plt.subplot(3,4,i+1)
    plt.imshow(image_batch[i].numpy().astype('uint8'))
    plt.title(dataset.class_names[labels_batch[i]])
    plt.axis('off')

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = 'training',
    seed = 123,
    image_size = image_size,
    batch_size = batch_size
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = 'validation',
    seed = 123,
    image_size = image_size,
    batch_size = batch_size
)

In [ ]:
num_classes = len(dataset.class_names)
num_classes

In [ ]:
model = keras.Sequential([
    keras.layers.Rescaling(1./255),
    layers.Conv2D(16,3,padding='same',activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32,3,padding='same',activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64,3,padding='same',activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
with tf.device('/device:GPU:0'):
    epochs = 50
    history = model.fit(
        train_ds,
        validation_data = val_ds,
        epochs = epochs
    )

In [ ]:
model.evaluate(val_ds)

In [ ]:
y_true = []
for images, labels in val_ds:
    y_true.extend(labels.numpy())

In [ ]:
classes_name = train_ds.class_names
classes_name

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred_all = []  # To store all predictions
y_true_all = []  # To store all true labels

for image_batch, label_batch in val_ds:
  y_pred = model.predict(image_batch)
  y_pred = np.argmax(y_pred, axis=1)

  y_pred_all.extend(y_pred)  # Accumulate predictions
  y_true_all.extend(label_batch.numpy())  # Accumulate true labels

print(classification_report(y_true_all, y_pred_all, target_names=classes_name))


In [ ]:
import random

def predict_random_images(model, dataset, num_images=20):
    """Predicts the class of random images from the dataset.

    Args:
        model: The trained model.
        dataset: The image dataset.
        num_images: The number of random images to predict.
    """

    # Get a list of image batches and labels from the dataset
    image_batches, label_batches = zip(*list(dataset.as_numpy_iterator()))

    # Flatten the batches into a single list
    all_images = [image for batch in image_batches for image in batch]
    all_labels = [label for batch in label_batches for label in batch]

    # Select random image indices
    random_indices = random.sample(range(len(all_images)), num_images)
    fig, axes = plt.subplots(4, 5, figsize=(20, 15))
    axes = axes.flatten()
    for idx, index in enumerate(random_indices):
        image = all_images[index]
        true_label = all_labels[index]

        # Reshape to fit model's expected input shape
        image = tf.expand_dims(image, 0)

        prediction = model.predict(image)
        predicted_class = np.argmax(prediction)

        # print(f"Image Index: {index}")
        # print(f"True Label: {classes_name[true_label]}")
        # print(f"Predicted Label: {classes_name[predicted_class]}")
        axes[idx].imshow(all_images[index].astype("uint8"))
        axes[idx].set_title(f"True: {classes_name[true_label]}\nPredicted: {classes_name[predicted_class]}")
        axes[idx].axis("off")
    plt.show()
    plt.tight_layout()

# Example usage (assuming 'model' and 'val_ds' are defined as in your previous code)
predict_random_images(model, dataset)

In [ ]:
# prompt: wirte code to save my model

model.save('my_model')
# or to save in a specific directory
# model.save('/content/drive/MyDrive/saved_models/my_model')


In [ ]:
from joblib import dump
dump(model, '/content/drive/MyDrive/A Tensor Flow/model_Tomato-leaf.joblib')